In [17]:
from finance.mongo.extraction import query_mongodb
from finance.params import *
import numpy as np
import pandas as pd


In [18]:
exchange_ls = EXCHANGE_LS
table_name = 'company_profile'


In [19]:
result = query_mongodb(exchange_ls=exchange_ls, table=table_name)

exchange {} - 80938


In [20]:
result_df = pd.DataFrame(result)
result_df

,_id,symbol,price,beta,volAvg,mktCap,lastDiv,range,changes,companyName,...,zip,dcfDiff,dcf,image,ipoDate,defaultImage,isEtf,isActivelyTrading,isAdr,isFund
0,657dc9f8b2cffe5b07e21e8d,1373.HK,1.7200,0.5190,653280,1.229432e+09,0.20,1.58-3.0,-0.0200,International Housewares Retail Company Limited,...,None,-13.48665,15.206650,https://financialmodelingprep.com/image-stock/...,2013-09-25,True,False,True,False,False
1,657dc9f9b2cffe5b07e21e8e,SSSSL,23.3966,0.0000,1214,1.070344e+08,1.50,22.28-25.0,0.1516,SuRo Capital Corp. 6.00% Notes due 2026,...,None,NaN,0.000000,https://financialmodelingprep.com/image-stock/...,2021-12-17,True,False,True,False,False
2,657dc9f9b2cffe5b07e21e8f,LVAC,10.4300,0.0000,44325,1.499312e+08,0.00,9.94-10.85,0.0000,LAVA Medtech Acquisition Corp.,...,02451,NaN,0.000000,https://financialmodelingprep.com/image-stock/...,2021-12-17,False,False,False,False,False
3,657dc9fab2cffe5b07e21e90,CPLP,13.8000,0.9040,38893,2.753293e+08,0.60,11.93-16.1,0.0500,Capital Product Partners L.P.,...,18537,-124.48770,138.809204,https://financialmodelingprep.com/image-stock/...,2007-03-30,True,False,True,False,False
4,657dc9fab2cffe5b07e21e91,MAGNUM.NS,48.9000,0.5460,72627,2.338735e+09,0.00,16.3-56.7,0.6000,Magnum Ventures Limited,...,201010,-185.45404,234.354044,https://financialmodelingprep.com/image-stock/...,2007-09-20,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80933,65833388570a04d5526854ba,SPRS,2.7000,1.3940,1273,1.503770e+07,0.00,2.5-3.68,0.0100,"Surge Components, Inc.",...,11729,-5.07622,7.776224,https://financialmodelingprep.com/image-stock/...,2010-03-26,False,False,True,False,False
80934,65833389570a04d5526854bb,MLRYY,0.9011,0.7100,0,1.435093e+09,0.00,0.9-1.25,0.0000,VK Company Limited,...,3107,NaN,0.000000,https://financialmodelingprep.com/image-stock/...,2012-04-20,False,False,False,False,False
80935,65833389570a04d5526854bc,SIMC,0.0001,-3638.4663,0,9.698020e+05,0.00,1.0E-4-0.01,0.0001,"Simclar, Inc.",...,33016,NaN,0.000000,https://financialmodelingprep.com/image-stock/...,1995-02-02,True,False,False,False,False
80936,6583338a570a04d5526854bd,SPKKY,15.9750,0.1760,53503,5.796241e+09,0.96,13.75-17.25,-0.1550,Spark New Zealand Limited,...,1010,NaN,0.000000,https://financialmodelingprep.com/image-stock/...,1991-07-17,False,False,True,False,False


In [25]:
result_df.groupby('exchangeShortName').count()

,_id,symbol,price,beta,volAvg,mktCap,lastDiv,range,changes,companyName,...,zip,dcfDiff,dcf,image,ipoDate,defaultImage,isEtf,isActivelyTrading,isAdr,isFund
exchangeShortName,,,,,,,,,,,,,,,,,,,,,
ASX,4160,4160,4156,4160,4160,4160,4160,4158,4156,4160,...,4098,1462,4160,4160,4160,4160,4160,4160,4160,4160
EURONEXT,3206,3206,3198,3206,3206,3206,3206,3190,3198,3206,...,3060,1140,3206,3206,3206,3206,3206,3206,3206,3206
HKSE,5224,5224,5222,5224,5224,5224,5224,5214,5222,5224,...,1250,1784,5224,5224,5224,5224,5224,5224,5224,5224
JPX,5298,5298,5298,5298,5298,5298,5298,5292,5298,5298,...,4954,3484,5298,5298,5298,5298,5298,5298,5298,5298
LSE,6910,6910,6908,6910,6910,6910,6910,6866,6908,6910,...,6514,3054,6910,6910,6910,6910,6910,6910,6910,6910
NASDAQ,12568,12568,12564,12568,12568,12566,12568,12388,12564,12568,...,11364,6202,12568,12568,12568,12568,12568,12568,12568,12568
NSE,4212,4212,4212,4212,4212,4212,4212,4208,4212,4210,...,3978,1980,4212,4212,4212,4212,4212,4212,4212,4212
NYSE,7183,7183,7183,7183,7183,7183,7183,7119,7183,7173,...,6286,4417,7183,7183,7183,7183,7183,7183,7183,7183
PNK,25849,25849,25849,25849,25849,25849,25849,25837,25849,25847,...,22832,5315,25849,25849,25849,25849,25849,25849,25849,25849


In [24]:
result_df[result_df['companyName']=='SolarEdge Technologies, Inc.'].T

,24305,30210,35618,52466
_id,657df79eb2cffe5b07e27d7e,657e02c0b2cffe5b07e2948f,657e0cf9b2cffe5b07e2a9af,657e2cafb2cffe5b07e2eb7f
symbol,0L7S.L,SEDG,0L7S.L,SEDG
price,96.93,97.12,96.93,97.12
beta,1.523,1.57,1.523,1.57
volAvg,5995,3173826,5995,3173826
mktCap,48316019.0,5517503744.0,48316019.0,5517503744.0
lastDiv,0.0,0.0,0.0,0.0
range,63.25-340.59,63.25-345.8,63.25-340.59,63.25-345.8
changes,2.65,0.8,2.65,0.8
companyName,"SolarEdge Technologies, Inc.","SolarEdge Technologies, Inc.","SolarEdge Technologies, Inc.","SolarEdge Technologies, Inc."


In [21]:
result_df.groupby('companyName').count()

,_id,symbol,price,beta,volAvg,mktCap,lastDiv,range,changes,currency,...,zip,dcfDiff,dcf,image,ipoDate,defaultImage,isEtf,isActivelyTrading,isAdr,isFund
companyName,,,,,,,,,,,,,,,,,,,,,
,373,373,373,373,373,373,373,371,373,373,...,0,49,373,373,373,373,373,373,373,373
"&Do Holdings Co.,Ltd.",2,2,2,2,2,2,2,2,2,2,...,2,0,2,2,2,2,2,2,2,2
01 Communique Laboratory Inc.,4,4,4,4,4,4,4,4,4,4,...,4,0,4,4,4,4,4,4,4,4
1&1 AG,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
"1-800-FLOWERS.COM, Inc.",2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Össur hf.,4,4,4,4,4,4,4,4,4,4,...,0,0,4,4,4,4,4,4,4,4
Österreichische Post AG,4,4,4,4,4,4,4,4,4,4,...,4,0,4,4,4,4,4,4,4,4
Ørsted A/S,4,4,4,4,4,4,4,4,4,4,...,4,0,4,4,4,4,4,4,4,4


In [26]:
result_df = result_df.drop_duplicates(subset=['companyName', 'symbol'])

In [27]:
result_df.dropna(subset='description', inplace=True)
result_df.dropna(subset=['sector', 'industry'], inplace=True)
result_df.drop_duplicates(subset='symbol', inplace=True)

/var/folders/3l/70h6qcrd72q0j5x4sscw7qn40000gn/T/ipykernel_69201/924731740.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df.dropna(subset='description', inplace=True)
/var/folders/3l/70h6qcrd72q0j5x4sscw7qn40000gn/T/ipykernel_69201/924731740.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df.dropna(subset=['sector', 'industry'], inplace=True)
/var/folders/3l/70h6qcrd72q0j5x4sscw7qn40000gn/T/ipykernel_69201/924731740.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

In [28]:
result_df.shape

(38258, 37)

In [29]:
result_description = result_df.to_dict('records')

In [32]:
import chromadb
from chromadb.config import Settings
from chromadb.utils import embedding_functions


chroma_client = chromadb.HttpClient(host="localhost", port = 8083, settings=Settings(allow_reset=True, anonymized_telemetry=False))
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

collection = chroma_client.create_collection(name="general_profile_mililm_l6_v2", embedding_function=sentence_transformer_ef)

In [9]:
import chromadb
from chromadb.utils import embedding_functions

chroma_client = chromadb.Client()
client = chromadb.PersistentClient(path="../chroma/")
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

/Users/michieldekoninck/.pyenv/versions/3.10.6/envs/finance/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [33]:
documents = [i['description'] for i in result_description]
ids = [i['symbol'] for i in result_description]
metadatas = [{'symbol':i['symbol'], 'exchangeShortName':i['exchangeShortName'], \
            'industry':i['industry'], 'sector':i['sector'], 'companyName':i['companyName']}\
        for i in result_description]

In [35]:
collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids)

In [38]:
results = collection.query(
    query_texts=["The increasing demand for AI technology benefits AMD, as it supplies high-quality GPUs for AI applications"],
    n_results=10
)
results['metadatas']

[[{'companyName': 'Advanced Micro Devices, Inc.',
   'exchangeShortName': 'NASDAQ',
   'industry': 'Semiconductors',
   'sector': 'Technology',
   'symbol': 'AMD'},
  {'companyName': 'Advanced Micro Devices, Inc.',
   'exchangeShortName': 'XETRA',
   'industry': 'Semiconductors',
   'sector': 'Technology',
   'symbol': 'AMD.DE'},
  {'companyName': 'NVIDIA Corporation',
   'exchangeShortName': 'XETRA',
   'industry': 'Semiconductors',
   'sector': 'Technology',
   'symbol': 'NVD.DE'},
  {'companyName': 'NVIDIA Corporation',
   'exchangeShortName': 'NASDAQ',
   'industry': 'Semiconductors',
   'sector': 'Technology',
   'symbol': 'NVDA'},
  {'companyName': 'NVIDIA Corporation',
   'exchangeShortName': 'XETRA',
   'industry': 'Semiconductors',
   'sector': 'Technology',
   'symbol': 'NVD.F'},
  {'companyName': 'Intel Corporation',
   'exchangeShortName': 'XETRA',
   'industry': 'Semiconductors',
   'sector': 'Technology',
   'symbol': 'INL.F'},
  {'companyName': 'Intel Corporation',
   'e

In [10]:
chroma_client.delete_collection('general_profile_mililm_l6_v2')

ValueError: Collection general_profile_mililm_l6_v2 does not exist.

In [11]:
collection = chroma_client.create_collection(name="general_profile_mililm_l6_v2", embedding_function=sentence_transformer_ef)

In [12]:
documents = [i['description'] for i in result_description]
ids = [i['symbol'] for i in result_description]
metadatas = [{'symbol':i['symbol'], 'exchangeShortName':i['exchangeShortName'], \
            'industry':i['industry'], 'sector':i['sector'], 'companyName':i['companyName']}\
        for i in result_description]

In [13]:
collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids)

In [14]:
chroma_client.get_collection(name="general_profile_mililm_l6_v2").peek() 

{'ids': ['0001.HK',
  '0002.HK',
  '0003.HK',
  '0004.HK',
  '0006.HK',
  '0007.HK',
  '0008.HK',
  '0009.HK',
  '0010.HK',
  '0011.HK'],
 'embeddings': [[0.04826970025897026,
   -0.04137176647782326,
   0.019185921177268028,
   -0.010492957197129726,
   0.021594269201159477,
   -0.027399128302931786,
   -0.03609631583094597,
   -0.015585441142320633,
   -0.013414482586085796,
   -0.0785144716501236,
   0.007142510265111923,
   -0.040814198553562164,
   0.006253519561141729,
   -0.007034194655716419,
   0.02683253400027752,
   -0.10852091759443283,
   0.029746733605861664,
   -0.1043149083852768,
   0.0017468748847022653,
   -0.09356296062469482,
   -0.012284748256206512,
   -0.07826907187700272,
   0.026388300582766533,
   -0.04375641420483589,
   -0.020737534388899803,
   -0.007294625975191593,
   0.06259078532457352,
   -0.020262939855456352,
   -0.006814217660576105,
   -0.04384222999215126,
   -0.0973985567688942,
   -0.006922221276909113,
   -0.0281873419880867,
   -0.00119616999

In [16]:
results = collection.query(
    query_texts=["solar panels and energy management"],
    n_results=10
)
results

{'ids': [['0L7S.L',
   'CSIQ',
   'SNRY',
   'SPWR',
   'SOEN',
   'SRGRF',
   'AMRC',
   'SNPW',
   'NEW.AX',
   '3868.HK']],
 'distances': [[0.8318488001823425,
   0.8589745759963989,
   0.8998551368713379,
   0.9123887419700623,
   0.9383266568183899,
   0.9596465826034546,
   0.97139972448349,
   0.9800097346305847,
   0.9865960478782654,
   0.988663375377655]],
 'metadatas': [[{'companyName': 'SolarEdge Technologies, Inc.',
    'exchangeShortName': 'LSE',
    'industry': 'Semiconductors & Semiconductor Equipment',
    'sector': 'Consumer Defensive',
    'symbol': '0L7S.L'},
   {'companyName': 'Canadian Solar Inc.',
    'exchangeShortName': 'NASDAQ',
    'industry': 'Solar',
    'sector': 'Technology',
    'symbol': 'CSIQ'},
   {'companyName': 'Solar Energy Initiatives, Inc.',
    'exchangeShortName': 'PNK',
    'industry': 'Shell Companies',
    'sector': 'Financial Services',
    'symbol': 'SNRY'},
   {'companyName': 'SunPower Corporation',
    'exchangeShortName': 'NASDAQ',
    

In [32]:
results['metadatas'][0]

[{'companyName': 'Energy Services of America Corporation',
  'exchangeShortName': 'NASDAQ',
  'industry': 'Engineering & Construction',
  'sector': 'Industrials',
  'symbol': 'ESOA'},
 {'companyName': 'American Electric Power Company, Inc.',
  'exchangeShortName': 'NASDAQ',
  'industry': 'Utilities—Regulated Electric',
  'sector': 'Utilities',
  'symbol': 'AEP'},
 {'companyName': 'American Electric Power Company, Inc.',
  'exchangeShortName': 'NASDAQ',
  'industry': 'Utilities—Regulated Electric',
  'sector': 'Utilities',
  'symbol': 'AEPPZ'},
 {'companyName': 'American Electric Power Company, Inc.',
  'exchangeShortName': 'LSE',
  'industry': 'Utilities',
  'sector': 'Utilities',
  'symbol': '0HEC.L'},
 {'companyName': 'American Battery Technology Company',
  'exchangeShortName': 'PNK',
  'industry': 'Other Industrial Metals & Mining',
  'sector': 'Basic Materials',
  'symbol': 'ABML'},
 {'companyName': 'The AES Corporation',
  'exchangeShortName': 'NYSE',
  'industry': 'Utilities—Div

In [52]:
results['documents']

[["CRISPR Therapeutics AG, a gene editing company, focuses on developing gene-based medicines for serious diseases using its proprietary Clustered Regularly Interspaced Short Palindromic Repeats (CRISPR)/CRISPR-associated protein 9 (Cas9) platform. Its CRISPR/Cas9 is a gene editing technology that allows for precise directed changes to genomic DNA. The company has a portfolio of therapeutic programs across a range of disease areas, including hemoglobinopathies, oncology, regenerative medicine, and rare diseases. The company's lead product candidate is CTX001, an ex vivo CRISPR gene-edited therapy for treating patients suffering from transfusion-dependent beta-thalassemia or severe sickle cell disease in which a patient's hematopoietic stem cells are engineered to produce high levels of fetal hemoglobin in red blood cells. It also develops CTX110, a donor-derived gene-edited allogeneic CAR-T investigational therapy targeting cluster of differentiation 19 positive malignancies; CTX120, a